In [1]:
import time
import sys
from itertools import permutations

class BacktrakingSolver:
    def __init__(self, initial_state):
        self.initial_state = initial_state
        self.solution = None
        self.start_time = None
        self.end_time = None
        self.memory_used = 0
        self.nodes_expanded = 0

    def is_valid_solution(self, state):
        for i in range(8):
            for j in range(i+1, 8):
                if abs(state[i] - state[j]) == abs(i - j):
                    return False
        return True

    def solve(self, time_limit=30, memory_limit=512):
        self.start_time = time.time()
        for perm in permutations(self.initial_state):
            self.memory_used += 1
            self.nodes_expanded += 1
            if self.is_valid_solution(perm):
                self.solution = perm
                break
            if time.time() - self.start_time > time_limit or sys.getsizeof(self) > memory_limit * 1024 * 1024:
                break
        self.end_time = time.time()

    def get_solution(self):
        return self.solution

    def get_elapsed_time(self):
        if self.start_time is None or self.end_time is None:
            return None
        return self.end_time - self.start_time

    def get_results(self):
        return {
            'Initial State': self.initial_state,
            'Solution': self.solution,
            'Time': self.get_elapsed_time(),
            'Count': self.nodes_expanded,
            'States': self.memory_used
        }


In [2]:
import sys
import time

class RBFSSolver:
    def __init__(self, initial_state):
        self.initial_state = initial_state
        self.solution = None
        self.start_time = None
        self.end_time = None
        self.nodes_expanded = 0

    def heuristic(self, state):
        pairs = 0
        for i in range(8):
            for j in range(i+1, 8):
                if abs(state[i] - state[j]) == abs(i - j):
                    pairs += 1
        return pairs

    def rbfs(self, state, f_limit, depth):
        if self.is_goal(state):
            return state, 0

        successors = self.get_successors(state)
        if not successors:
            return None, sys.maxsize

        for s in successors:
            s.append(depth + 1 + max(self.heuristic(s), f_limit))

        min_successor = None
        while True:
            successors.sort(key=lambda x: x[2])
            best = successors[0]
            if best[2] > f_limit:
                return None, best[2]
            alternative = successors[1][2] if len(successors) > 1 else sys.maxsize
            result, new_f = self.rbfs(best, min(alternative, f_limit), depth + 1)
            best[2] = new_f
            if result is not None:
                return result, new_f

    def is_goal(self, state):
        for i in range(8):
            for j in range(i+1, 8):
                if abs(state[i] - state[j]) == abs(i - j):
                    return False
        return True

    def get_successors(self, state):
        successors = []
        for i in range(8):
            for j in range(1, 8):
                new_state = list(state)
                new_state[i] = (new_state[i] + j) % 8
                successors.append(new_state)
        return successors

    def solve(self, time_limit=30):
        self.start_time = time.time()
        self.solution, _ = self.rbfs(self.initial_state, sys.maxsize, 0)
        self.end_time = time.time()

    def get_solution(self):
        return self.solution

    def get_elapsed_time(self):
        if self.start_time is None or self.end_time is None:
            return None
        return self.end_time - self.start_time

    def get_results(self):
        return {
            'Initial State': self.initial_state,
            'Solution': self.solution,
            'Time': self.get_elapsed_time(),
            'Nodes Expanded': self.nodes_expanded
        }


In [106]:
import random
import pandas as pd

def print_RBFS(num_experiments, memori):
    results = []

    for i in range(num_experiments):
      initial_state = memori[i]
      solver = RBFSSolver(initial_state)
      solver.solve()
      results.append(solver.get_results())

    results_df = pd.DataFrame(results)
    print("Results of each experiment for RBFS:")
    print(results_df)

def print_backtriceng(num_experiments, memori):
    results = []

    for i in range(num_experiments):
      initial_state = memori[i]
      solver = BacktrakingSolver(initial_state)
      solver.solve()
      results.append(solver.get_results())

    results_df = pd.DataFrame(results)
    print("Results of each experiment for BACKTRAKING:")
    print(results_df)

    avg_results = results_df.mean()
    print("\nAverage results:")
    print(avg_results)

def start(num_experiments=20):
    initial_state = [0, 1, 2, 3, 4, 5, 6, 7]
    memori = []

    for _ in range(num_experiments):
        shuffled_state = initial_state.copy()
        random.shuffle(shuffled_state)
        memori.append(shuffled_state)

    print_backtriceng(num_experiments, memori)
    # print_RBFS(num_experiments, memori)

start()

Results of each experiment for BACKTRAKING:
               Initial State                  Solution      Time  Count  \
0   [3, 4, 5, 6, 2, 0, 7, 1]  (3, 5, 0, 4, 1, 7, 2, 6)  0.002999   1104   
1   [4, 3, 5, 7, 1, 6, 2, 0]  (4, 7, 3, 0, 6, 1, 5, 2)  0.002409   1551   
2   [0, 7, 5, 6, 3, 1, 4, 2]  (0, 5, 7, 2, 6, 3, 1, 4)  0.001994    817   
3   [5, 6, 7, 3, 0, 2, 4, 1]  (5, 7, 1, 3, 0, 6, 4, 2)  0.001994   1352   
4   [5, 0, 7, 4, 1, 2, 3, 6]  (5, 0, 4, 1, 7, 2, 6, 3)  0.000000    146   
5   [6, 7, 0, 5, 2, 3, 1, 4]  (6, 0, 2, 7, 5, 3, 1, 4)  0.001997    961   
6   [2, 1, 0, 5, 4, 3, 6, 7]  (2, 0, 6, 4, 7, 1, 3, 5)  0.001993   1268   
7   [6, 2, 5, 4, 0, 7, 1, 3]  (6, 2, 0, 5, 7, 4, 1, 3)  0.000000    247   
8   [5, 3, 4, 1, 2, 7, 6, 0]  (5, 3, 1, 7, 4, 6, 0, 2)  0.000000    172   
9   [5, 6, 4, 3, 2, 1, 0, 7]  (5, 3, 6, 0, 2, 4, 1, 7)  0.002993   1519   
10  [6, 3, 1, 2, 5, 7, 4, 0]  (6, 3, 1, 7, 5, 0, 2, 4)  0.000000     59   
11  [5, 6, 1, 3, 7, 0, 4, 2]  (5, 1, 6, 0, 3, 7, 4, 2)  

C:\Users\1slav\AppData\Local\Temp\ipykernel_40604\3476764613.py:30: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  avg_results = results_df.mean()
